<a href="https://colab.research.google.com/github/Suffragette/github-slideshow/blob/master/%CE%95%CF%81%CE%B3%CE%B1%CF%83%CF%84%CE%B7%CF%81%CE%B9%CE%B1%CE%BA%CE%AE_%CE%86%CF%83%CE%BA%CE%B7%CF%83%CE%B7_%CE%A4%CE%B5%CF%87%CE%BD%CE%B9%CE%BA%CE%AD%CF%82_%CE%94%CE%B9%CE%B1%CF%87%CE%B5%CE%AF%CF%81%CE%B9%CF%83%CE%B7%CF%82_%CE%BA%CE%B1%CE%B9_%CE%95%CE%BE%CF%8C%CF%81%CF%85%CE%BE%CE%B7%CF%82_%CE%B3%CE%B9%CE%B1_%CE%94%CE%B5%CE%B4%CE%BF%CE%BC%CE%AD%CE%BD%CE%B1_%CE%9C%CE%B5%CE%B3%CE%AC%CE%BB%CE%BF%CF%85_%CE%8C%CE%B3%CE%BA%CE%BF%CF%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install pyspark==3.1.2
!pip install spark-nlp==3.3.1

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

import findspark
findspark.init()

from pyspark.sql import SparkSession
import sparknlp

spark = SparkSession.builder \
    .appName("Spark NLP") \
    .master("local[4]") \
    .config("spark.driver.memory","16G") \
    .config("spark.driver.maxResultSize", "0") \
    .config("spark.kryoserializer.buffer.max", "2000M") \
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:3.3.1") \
    .getOrCreate()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)


Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Get:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Fetched 336 kB in 1s (237 kB/s)
Reading package lists... Done
Spark NLP version:  3.3.1
Apache Spark version:  3.1.2


In [56]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [57]:
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder.getOrCreate()

# Load the data
# Load the data
labels = spark.read.csv("/content/drive/MyDrive/Chatgpt/Τεχνικές Διαχείρισης και Εξόρυξης για  Δεδομένα Μεγάλου Όγκου/Medical-Abstracts-TC-Corpus-main/medical_tc_labels.csv", inferSchema=True, header=True)
train = spark.read.csv("/content/drive/MyDrive/Chatgpt/Τεχνικές Διαχείρισης και Εξόρυξης για  Δεδομένα Μεγάλου Όγκου/Medical-Abstracts-TC-Corpus-main/medical_tc_train.csv", inferSchema=True, header=True)
test = spark.read.csv("/content/drive/MyDrive/Chatgpt/Τεχνικές Διαχείρισης και Εξόρυξης για  Δεδομένα Μεγάλου Όγκου/Medical-Abstracts-TC-Corpus-main/medical_tc_test.csv", inferSchema=True, header=True)


# Show the first few rows of the DataFrame
data.show()


+---------------+--------------------+
|condition_label|    medical_abstract|
+---------------+--------------------+
|              5|Tissue changes ar...|
|              1|Neuropeptide Y an...|
|              2|Sexually transmit...|
|              1|Lipolytic factors...|
|              3|Does carotid rest...|
|              3|The shoulder in m...|
|              2|The management of...|
|              4|Pharmacomechanica...|
|              5|Color Doppler dia...|
|              5|Noninvasive diagn...|
|              4|Sodium sensitive ...|
|              1|Imaging bone tumo...|
|              5|Closure of a bron...|
|              1|Utility of frozen...|
|              4|Antihypertensive ...|
|              2|Gallbladder perfo...|
|              1|Left ventricular ...|
|              4|Tongue ischemia f...|
|              5|In vitro and in v...|
|              1|Chondrosarcoma of...|
+---------------+--------------------+
only showing top 20 rows



In [58]:
print(data.columns)


['condition_label', 'medical_abstract']


In [62]:
from sparknlp.base import DocumentAssembler, Finisher
from sparknlp.annotator import Tokenizer, Normalizer, StopWordsCleaner

# Assemble the text column into a Document type column
document_assembler = DocumentAssembler() \
    .setInputCol("medical_abstract") \
    .setOutputCol("document")

# Tokenize the document
tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

# Normalize the tokens (remove punctuation and convert to lowercase)
normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normalized")

# Remove stopwords
stopwords_cleaner = StopWordsCleaner() \
    .setInputCols(["normalized"]) \
    .setOutputCol("clean_tokens") \
    .setCaseSensitive(False)

# Finish the preprocessing (convert the result back to a string column)
finisher = Finisher() \
    .setInputCols(["clean_tokens"]) \
    .setOutputCols(["clean_text"]) \
    .setOutputAsArray(False) \
    .setCleanAnnotations(False)

# Define the stages of the pipeline
stages = [document_assembler, tokenizer, normalizer, stopwords_cleaner, finisher]

# Create a pipeline
from pyspark.ml import Pipeline
pipeline = Pipeline().setStages(stages)

# Fit the pipeline to the training data
model = pipeline.fit(train)

# Transform the training and test data
train_clean = model.transform(train)
test_clean = model.transform(test)

# Fit the pipeline to the test data and transform it
test_clean = model.transform(test)

# Show the first few rows of the cleaned training data
train_clean.show()

# Show the first few rows of the cleaned test data
test_clean.show()



+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|condition_label|    medical_abstract|            document|               token|          normalized|        clean_tokens|          clean_text|
+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|              5|Tissue changes ar...|[{document, 0, 10...|[{token, 0, 5, Ti...|[{token, 0, 5, Ti...|[{token, 0, 5, Ti...|Tissue@changes@ar...|
|              1|Neuropeptide Y an...|[{document, 0, 12...|[{token, 0, 11, N...|[{token, 0, 11, N...|[{token, 0, 11, N...|Neuropeptide@Y@ne...|
|              2|Sexually transmit...|[{document, 0, 17...|[{token, 0, 7, Se...|[{token, 0, 7, Se...|[{token, 0, 7, Se...|Sexually@transmit...|
|              1|Lipolytic factors...|[{document, 0, 10...|[{token, 0, 8, Li...|[{token, 0, 8, Li...|[{token, 0, 8, Li...|Lipolytic@fact

In [ ]:
from sparknlp.annotator import WordEmbeddingsModel

# Load the pre-trained word embeddings model
embeddings = WordEmbeddingsModel.pretrained('glove_100d', 'en') \
    .setInputCols(["document", "token"]) \
    .setOutputCol("embeddings")

# Add the embeddings to the stages of the pipeline
stages = [document_assembler, tokenizer, normalizer, stopwords_cleaner, embeddings, finisher]

# Create a pipeline
from pyspark.ml import Pipeline
pipeline = Pipeline().setStages(stages)

# Fit the pipeline to the data
model = pipeline.fit(data)

# Transform the data
data_clean = model.transform(data)

# Show the first few rows of the cleaned data
data_clean.show()


glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]
+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|condition_label|    medical_abstract|            document|               token|          normalized|        clean_tokens|          embeddings|          clean_text|
+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|              3|Obstructive sleep...|[{document, 0, 15...|[{token, 0, 10, O...|[{token, 0, 10, O...|[{token, 0, 10, O...|[{word_embeddings...|Obstructive@sleep...|
|              5|Neutrophil functi...|[{document, 0, 16...|[{token, 0, 9, Ne...|[{token, 0, 9, Ne...|[{token, 0, 9, Ne...|[{word_embeddings...|Neutrophil@functi...|
|              5|A phase II study ...|[{document, 0, 18...|[{token, 0, 0, A,..

In [ ]:
from sparknlp.annotator import SentenceEmbeddings

# Create a SentenceEmbeddings object
sentence_embeddings = SentenceEmbeddings() \
    .setInputCols(["document", "embeddings"]) \
    .setOutputCol("sentence_embeddings") \
    .setPoolingStrategy("AVERAGE")

# Add the sentence_embeddings to the stages of the pipeline
stages = [document_assembler, tokenizer, normalizer, stopwords_cleaner, embeddings, sentence_embeddings, finisher]

# Create a pipeline
from pyspark.ml import Pipeline
pipeline = Pipeline().setStages(stages)

# Fit the pipeline to the data
model = pipeline.fit(data)

# Transform the data
data_clean = model.transform(data)

# Show the first few rows of the cleaned data
data_clean.show()


+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|condition_label|    medical_abstract|            document|               token|          normalized|        clean_tokens|          embeddings| sentence_embeddings|          clean_text|
+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|              3|Obstructive sleep...|[{document, 0, 15...|[{token, 0, 10, O...|[{token, 0, 10, O...|[{token, 0, 10, O...|[{word_embeddings...|[{sentence_embedd...|Obstructive@sleep...|
|              5|Neutrophil functi...|[{document, 0, 16...|[{token, 0, 9, Ne...|[{token, 0, 9, Ne...|[{token, 0, 9, Ne...|[{word_embeddings...|[{sentence_embedd...|Neutrophil@functi...|
|              5|A phase II study ...|[{document, 0, 18...|[{token, 0,

In [40]:
data_clean.select("finished_sentence_embeddings").show(truncate=False)


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [41]:
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, FloatType

def flatten_array_of_arrays(data):
    return [item for sublist in data for item in sublist]

flatten_udf = udf(flatten_array_of_arrays, ArrayType(FloatType()))

data_clean = data_clean.withColumn("flattened_embeddings", flatten_udf("finished_sentence_embeddings"))


In [ ]:
from pyspark.sql import Row

# Convert DataFrame to RDD
train_data_rdd = train_data.rdd

# Define a function to flatten the array
def flatten_array(row):
    flattened_array = [item for sublist in row.sentence_embeddings_vec for item in sublist]
    return Row(**{"condition_label": row.condition_label, "sentence_embeddings_vec": flattened_array})

# Apply the function to the RDD
train_data_rdd = train_data_rdd.map(flatten_array)

# Convert RDD back to DataFrame
train_data = spark.createDataFrame(train_data_rdd)

# Now you can use the VectorAssembler
assembler = VectorAssembler(
    inputCols=["sentence_embeddings_vec"],
    outputCol="features")

train_data = assembler.transform(train_data)


In [44]:
from pyspark.sql import Row

# Convert DataFrame to RDD
train_data_rdd = train_data.rdd

# Define a function to flatten the array
def flatten_array(row):
    flattened_array = [item for sublist in row.sentence_embeddings_vec for item in sublist]
    return Row(**{"condition_label": row.condition_label, "sentence_embeddings_vec": flattened_array})

# Apply the function to the RDD
train_data_rdd = train_data_rdd.map(flatten_array)

# Convert RDD back to DataFrame
train_data = spark.createDataFrame(train_data_rdd)

# Now you can use the VectorAssembler
assembler = VectorAssembler(
    inputCols=["sentence_embeddings_vec"],
    outputCol="features")

train_data = assembler.transform(train_data)


PythonException: ignored

In [43]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=["sentence_embeddings_vec"],
    outputCol="features")

train_data = assembler.transform(train_data)
test_data = assembler.transform(test_data)


PythonException: ignored

In [42]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=["flattened_embeddings"],
    outputCol="features")

train_data = assembler.transform(train_data)
test_data = assembler.transform(test_data)


IllegalArgumentException: ignored

In [39]:
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf

# Define a UDF to convert array of floats to vector
list_to_vector_udf = udf(lambda l: Vectors.dense(l), VectorUDT())

# Apply the UDF to the finished_sentence_embeddings column
data_clean = data_clean.withColumn("sentence_embeddings_vec", list_to_vector_udf("finished_sentence_embeddings"))

# Now split the data
train_data, test_data = data_clean.randomSplit([0.8, 0.2], seed=12345)

# Now you can use the VectorAssembler
assembler = VectorAssembler(
    inputCols=["sentence_embeddings_vec"],
    outputCol="features")

train_data = assembler.transform(train_data)
test_data = assembler.transform(test_data)


PythonException: ignored

In [36]:
from sparknlp.base import EmbeddingsFinisher

# Create an EmbeddingsFinisher
finisher = EmbeddingsFinisher() \
    .setInputCols(["sentence_embeddings"]) \
    .setOutputCols(["finished_sentence_embeddings"]) \
    .setOutputAsVector(True) \
    .setCleanAnnotations(False)

# Transform the data
data_clean = finisher.transform(data_clean)

# Now you can use the VectorAssembler
assembler = VectorAssembler(
    inputCols=["finished_sentence_embeddings"],
    outputCol="features")

train_data = assembler.transform(train_data)
test_data = assembler.transform(test_data)


IllegalArgumentException: ignored

In [35]:
from sparknlp.functions import map_annotations_col
from pyspark.sql.functions import udf
from pyspark.ml.linalg import VectorUDT, Vectors

# Define a UDF to flatten the sentence_embeddings column
flatten = udf(lambda v: Vectors.dense(v), VectorUDT())

# Apply the UDF to the sentence_embeddings column
data_clean = data_clean.withColumn("sentence_embeddings", flatten(map_annotations_col("sentence_embeddings")))

# Now you can use the VectorAssembler
assembler = VectorAssembler(
    inputCols=["sentence_embeddings"],
    outputCol="features")

train_data = assembler.transform(train_data)
test_data = assembler.transform(test_data)


TypeError: ignored

In [34]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

# Convert the sentence embeddings to a format that can be used by the ML model
assembler = VectorAssembler(
    inputCols=["sentence_embeddings"],
    outputCol="features")

train_data = assembler.transform(train_data)
test_data = assembler.transform(test_data)

# Create a logistic regression model
lr = LogisticRegression(featuresCol='features', labelCol='condition_label')

# Train the model
lr_model = lr.fit(train_data)

# Make predictions on the test data
predictions = lr_model.transform(test_data)

# Evaluate the model
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(
    labelCol="condition_label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test set accuracy = " + str(accuracy))


IllegalArgumentException: ignored